# Cell/particle Counting and scoring stained objects

This notebook demonstrates how to process plates associated to the paper ['Integration of biological data by kernels on graph nodes allows prediction of new genes involved in mitotic chromosome condensation.'](http://dx.doi.org/10.1091/mbc.E13-04-0221).
We use the [Cell/particle counting, and scoring the percentage of stained objects](http://cellprofiler.org/examples/#PercentPositive) pipeline.

### Import Packages

In [ ]:
# %tb

# Import Cell Profiler Dependencies
import cellprofiler
import cellprofiler.preferences as cpprefs
import cellprofiler.module as cpm
import cellprofiler.pipeline as cpp
cpprefs.set_headless()

# Inject Image module used to inject OMERO image planes into Cell Profiler Pipeline
from cellprofiler.modules.injectimage import InjectImage

# Import OMERO Python BlitzGateway
import omero
from omero.gateway import BlitzGateway

# Import Numpy
import numpy as np

# Import Python System Packages
import os
import tempfile
import pandas
import warnings

### Set Cell Output Directory

In [ ]:
new_output_directory = os.path.normcase(tempfile.mkdtemp())
cpprefs.set_default_output_directory(new_output_directory)

### OMERO Credentials

In [ ]:
HOST = 'outreach.openmicroscopy.org'
# To be modified
USERNAME = 'username'
PASSWORD = 'password'
PORT = 4064
client = omero.client(host=HOST, port=PORT)
client.createSession(USERNAME, PASSWORD)
conn = BlitzGateway(client_obj=client)

### Fetch OMERO Plate that contains the Images to be analysed

In [ ]:
# To be modified
plate_id = 422
plate = conn.getObject("Plate", plate_id)
print 'Plate Name: ', plate.getName()

In [ ]:
from os.path import expanduser
home = expanduser("~")
pipeline = cpp.Pipeline()
pipeline.load(home+"/notebooks/CellProfiler/pipelines/ExamplePercentPositive.cppipe")
for module in pipeline.modules():
    print  module.module_name

### Run Cell Profiler Pipeline on a plate

In [ ]:
warnings.filterwarnings('ignore')

Nuclei = pandas.DataFrame()
Image = pandas.DataFrame()

for well in plate.listChildren():
    
    index = well.countWellSample()
    for index in xrange(0, index):
        image = well.getImage(index)
        pixels = image.getPrimaryPixels()
        size_c = image.getSizeC()
        pipeline_copy = pipeline.copy()
        for c in range(0, size_c):
        
            plane = pixels.getPlane(0, c, 0)
            image_name = image.getName()
            
            if c == 0:
                image_name = 'OrigBlue'
            if c == 1:
                image_name = 'OrigGreen'
            if c == 2:
                image_name = 'OrigRed'
            
            inject_image_module = InjectImage(image_name, plane)
            inject_image_module.set_module_num(1)
            pipeline_copy.add_module(inject_image_module)
            
        for i in [1, 2, 3, 4]:
            pipeline_copy.remove_module(pipeline_copy.modules()[size_c].module_num)
        
        m = pipeline_copy.run()
        
        nuc_csv = pandas.read_csv(new_output_directory + '/Nuclei.csv')
        nuc_csv.ImageNumber = image.getName()
        Nuclei = Nuclei.append(nuc_csv)
        
        image_csv = pandas.read_csv(new_output_directory + '/Image.csv')
        image_csv.ImageNumber = image.getName()
        Image = Image.append(image_csv)
        
    for module in pipeline_copy.modules():
        print  module.module_name

## Results obtained as CSV from Cell Profiler

### Measurements (Nuclei)

In [ ]:
Nuclei.head()

### Images (Summary)

In [ ]:
Image.head()

### Push Results back to OMERO

In [ ]:
import glob

for file_to_upload in glob.glob(new_output_directory + '/*.csv'):
    
    if file_to_upload.endswith('Nuclei.csv'):
        Nuclei.to_csv(file_to_upload)
    elif file_to_upload.endswith('Image.csv'):
        Image.to_csv(file_to_upload)
    
    # create the original file and file annotation (uploads the file etc.)
    namespace = "training.meeting.demo"
    print "\nCreating an OriginalFile and FileAnnotation"
    file_ann = conn.createFileAnnfromLocalFile(file_to_upload, mimetype="text/plain", ns=namespace, desc=None)
    print "Attaching FileAnnotation to Plate: ", "File ID:", file_ann.getId(), \
        ",", file_ann.getFile().getName(), "Size:", file_ann.getFile().getSize()
    dataset.linkAnnotation(file_ann)

<br>
#### Close the connection to the OMERO server

In [ ]:
conn.close()